In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.brain.internal.models as fbm
import numpy as np

model = fbm.load_model("simple-resnet-cifar10")
#np.random.seed(41)
def find_unique(dataset_dir,sub_thresh=0.01,subset_files=None):
	try:
		fo.core.dataset.delete_non_persistent_datasets()
	except:
		pass
	dataset = fo.Dataset.from_dir(dataset_dir=dataset_dir,dataset_type=fo.types.ImageDirectory,name='temp1',)
	if subset_files is not None:
		for sample in dataset:
			if sample.filename not in subset_files:
				dataset.delete_samples(sample.id)
	embeddings = dataset.compute_embeddings(model, batch_size=16)
	print("shape of embedding is:",embeddings.shape)
	results = fob.compute_similarity(dataset, embeddings=embeddings, brain_key="img_sim")
	uniques,threshes=[],[]
	for thresh in np.arange(0.01,1,10):
		try:
			results.find_duplicates(thresh=thresh)
			print('Unique samples found:'+ str(len(results.unique_ids)) + ' at thresh='+str(thresh))
			uniques.append(len(results.unique_ids))
			threshes.append(thresh)
			if len(uniques)>1:
				if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
					break
		except:
			pass
	arr=np.round(np.diff(np.flip(uniques))/len(dataset),4)>sub_thresh
	print("the values in arr:",arr)
	cont=len(uniques)-1
	found=False
	#
	print("test")
	for j in range(cont,0,-1):
		for i in range(len(arr)):
			if i+j==len(arr)+1:
				break
			if arr[i:i+j].all():
				idx=i
				found=True
				break
		if found:
			cont=j
			break
	#idx=0	
	try:
		idx
	except NameError:		
		if len(uniques)>1:
			if ((uniques[-2]-uniques[-1])/len(dataset))<(sub_thresh*0.5):
				idx = len(uniques)-1
				print("length of uniques is",len(uniques))
					
	thresh=np.flip(threshes)[idx]
	results.find_duplicates(thresh=thresh)
	print('\nThreshold fixed at:'+str(thresh)+' at continuity count of '+str(cont))
	print('Total Unique samples collected:', len(results.unique_ids))
	print('Imagenames of Unique samples are', results.unique_ids) #this is the list of filenames
	unique_fname=[sample.filename for sample in dataset if sample.id in results.unique_ids]
	dataset.delete()
	return unique_fname




In [2]:
if __name__=='__main__':
	dataset_dir='C://Users//320199927//Desktop//SUBHASH ALL IMPORTANT INFO//Images4exp_110' # Provide path to image folder
	unique_files=find_unique(dataset_dir,sub_thresh=0.01,subset_files=None)

 100% |█████████████████| 110/110 [169.3ms elapsed, 0s remaining, 668.8 samples/s] 
 100% |█████████████████| 110/110 [4.0s elapsed, 0s remaining, 27.7 samples/s]      
shape of embedding is: (110, 512)
Generating index...
Index complete
Computing duplicate samples...
Duplicates computation complete
Unique samples found:52 at thresh=0.01
the values in arr: []
test


UnboundLocalError: local variable 'idx' referenced before assignment